In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import evaluate
from pathlib import Path

NOTEBOOK_DIR = Path("/".join(__vsc_ipynb_file__.split("/")[:-1]))
print(NOTEBOOK_DIR)

/home/kyre/repos/llm-fine-tuning


In [12]:
# https://huggingface.co/datasets/gretelai/synthetic_text_to_sql
dataset = load_dataset("gretelai/synthetic_text_to_sql")

print("Available data subsets:", dataset.keys())
print("Features: ")
for k, v in dataset["train"][0].items():
    print(f"---> {k:30}: {v}")


Available data subsets: dict_keys(['train', 'test'])
Features: 
---> id                            : 5097
---> domain                        : forestry
---> domain_description            : Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.
---> sql_complexity                : single join
---> sql_complexity_description    : only one join (specify inner, outer, cross)
---> sql_task_type                 : analytics and reporting
---> sql_task_type_description     : generating reports, dashboards, and analytical insights
---> sql_prompt                    : What is the total volume of timber sold by each salesperson, sorted by salesperson?
---> sql_context                   : CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_i

In [13]:
train_df = pd.DataFrame(dataset["train"])
test_df = pd.DataFrame(dataset["train"])

In [14]:
from tqdm import tqdm

def check_missing_data(dataset, columns):
    for split in dataset.keys():
        print(f"Checking '{split}' split...")
        for column in tqdm(columns):
            if column not in dataset[split].column_names:
                print(f"  Column '{column}' not found in the dataset!")
                continue
            missing_count = sum(1 for example in dataset[split] if not example[column] or example[column].strip() == "")
            print(f"  {column}: {missing_count} missing or empty entries")
        print()

print("Dataset columns:", dataset["train"].column_names)
check_missing_data(dataset, columns=["sql_prompt", "sql"])

Dataset columns: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation']
Checking 'train' split...


 50%|█████     | 1/2 [00:07<00:07,  7.31s/it]

  sql_prompt: 0 missing or empty entries


100%|██████████| 2/2 [00:14<00:00,  7.19s/it]


  sql: 0 missing or empty entries

Checking 'test' split...


 50%|█████     | 1/2 [00:00<00:00,  2.06it/s]

  sql_prompt: 0 missing or empty entries


100%|██████████| 2/2 [00:00<00:00,  2.26it/s]

  sql: 0 missing or empty entries



In [ ]:
def normalize_text(text):
    text = text.lower().strip().replace("\n", " ")
    return text

tokenizer = AutoTokenizer.from_pretrained("t5-large")

In [ ]:
def preprocess(example):
    input_text = "Translate to SQL: " + normalize_text(example["sql_prompt"])
    output_text = normalize_text(example["sql"])

    model_inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(output_text, truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess, remove_columns=dataset["train"].column_names)

available_splits = dataset.keys()
train_ds = tokenized_ds["train"]
validation_ds = tokenized_ds["test"] if "test" in available_splits else None
test_ds = tokenized_ds["test"] if "test" in available_splits else None

Map: 100%|██████████| 5851/5851 [00:02<00:00, 2664.39 examples/s]


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-large")

training_args = Seq2SeqTrainingArguments(
    output_dir="./sql_model",
    eval_strategy="epoch",  # Changed from evaluation_strategy
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipykernel_956164/3438895915.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
def evaluate_model(test_dataset):
    metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_labels = [[label] for label in decoded_labels]
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    results = trainer.evaluate(eval_dataset=test_dataset, compute_metrics=compute_metrics)
    return results

if test_ds:
    test_results = evaluate_model(test_ds)
    print(test_results)

In [ ]:
def driver(question):
    inputs = tokenizer("Translate to SQL: " + question, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(driver("Find all customers who ordered in 2023"))